<a href="https://colab.research.google.com/github/saotomryo/Use_MMOCR/blob/main/Use_MMOCR_seg_r31_1by16_fpnocr_toy_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import json
from skimage import measure
import numpy as np
print(torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device = ", device)

1.11.0+cu113
device =  cuda


## 環境準備 MMCV MMOCRのインストール

In [2]:
#!pip install mmcv-full==1.3.17
#!pip install mmcv-full
!pip install https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/mmcv_full-1.4.7-cp37-cp37m-manylinux1_x86_64.whl

     |████████████████████████████████| 37.6 MB 16.2 MB/s 
     |████████████████████████████████| 190 kB 9.7 MB/s 


In [3]:
# Install mmdetection
!pip install mmdet

     |████████████████████████████████| 1.4 MB 7.5 MB/s 


In [4]:
HOME_PATH = "/content"

In [5]:

!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
!pip install -r requirements/build.txt
!pip install -v -e .  # or "python setup.py develop"

ストリーミング出力は最後の 5000 行に切り捨てられました。
    Skipping link: none of the wheel's tags (cp36-cp36m-win_amd64) are compatible (run pip debug --verbose to show compatible tags): https://files.pythonhosted.org/packages/dc/18/e69ef84530360c2d39db51acb4cc0012990f27fb1fa7542dac45ad30e7ab/numpy-1.18.5-cp36-cp36m-win_amd64.whl#sha256=b03b2c0badeb606d1232e5f78852c102c0a7989d3a534b3129e7856a52f3d161 (from https://pypi.org/simple/numpy/) (requires-python:>=3.5)
    Skipping link: none of the wheel's tags (cp37-cp37m-macosx_10_9_x86_64) are compatible (run pip debug --verbose to show compatible tags): https://files.pythonhosted.org/packages/3e/00/0266fefaafb839760d5b25b884375b2ab0f842ebe138ee6c1ef807af44bb/numpy-1.18.5-cp37-cp37m-macosx_10_9_x86_64.whl#sha256=a7acefddf994af1aeba05bbbafe4ba983a187079f125146dc5859e6d817df824 (from https://pypi.org/simple/numpy/) (requires-python:>=3.5)
    Skipping link: none of the wheel's tags (cp37-cp37m-manylinux1_i686) are compatible (run pip debug --verbose to show compa

## コンフィグファイルの編集

In [6]:
%cd /content/mmocr

/content/mmocr


In [20]:
from mmcv import Config
cfg = Config.fromfile('/content/mmocr/configs/textrecog/sar/sar_r31_parallel_decoder_toy_dataset.py')


## アノテーションファイルの変換

　toy_datasetのアノテーションファイルはファイル名（半角スペース)対象文字列で、テキストファイルとして作成します。

　ファイル名 対象の文字列

作成したテキストファイルをlmdbフォルダに形式変換して保存します。



In [21]:
## アノテーションフォルダの設定
#　元になるテキストファイル
label_path = #ラベルファイルのパス
lmdb_path = #ラベル変換出力したファイルのパス

# 画像ファイルのパス
img_path = 画像フォルダのパス

# ルートフォルダ
root_path = ルートフォルダのパス

# 作業フォルダ（学習時のモデル出力先（エポック毎に出力))
work_path = 作業フォルダのパス


In [15]:
!python /content/mmocr/tools/data/utils/txt2lmdb.py \
    -i $label_path \
    -o $lmdb_path

/content/drive/MyDrive/signate/Toppan2/test_ocr/train_text/label.lmdb already exist, delete or not? [Y/n]
y
create database /content/drive/MyDrive/signate/Toppan2/test_ocr/train_text/label.lmdb
[00:11:06-00:11:06], processing [0-9] / 9
done


In [23]:
from mmdet.apis import set_random_seed

# コンフィグを変更する
# sar

# 作業フォルダ
cfg.work_dir = work_path

# rootフォルダ
cfg.root = root_path

# アノテーションファイルのパス
cfg.train_anno_file1 = label_path
cfg.train_anno_file2 = lmdb_path
cfg.img_prefix = img_path
cfg.train1.ann_file = label_path
cfg.train1.img_prefix = img_path
cfg.train2.ann_file = label_path
cfg.train2.img_prefix = img_path
cfg.test.ann_file = lmdb_path
cfg.test.img_prefix = img_path
cfg.test_anno_file1 = lmdb_path
cfg.train_list[0].ann_file = label_path
cfg.train_list[0].img_prefix = img_path
cfg.train_list[1].ann_file = lmdb_path
cfg.train_list[1].img_prefix = img_path
cfg.test_list[0].ann_file = lmdb_path
cfg.test_list[0].img_prefix = img_path
cfg.data.test.datasets[0].ann_file = lmdb_path
cfg.data.test.datasets[0].img_prefix = img_path
cfg.data.train.datasets[0].ann_file = label_path
cfg.data.train.datasets[0].img_prefix = img_path
cfg.data.train.datasets[1].ann_file = lmdb_path
cfg.data.train.datasets[1].img_prefix = img_path
cfg.data.val.datasets[0].ann_file = lmdb_path
cfg.data.val.datasets[0].img_prefix = img_path


# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.001 / 8
cfg.lr_config.warmup = None
# Choose to log training results every 40 images to reduce the size of log file. 
cfg.log_config.interval = 40

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


# 編集したコンフィグをファイル出力
cfg.dump('/content/seg_r31_1by16_fpnocr_toy_dataset.py')


Config:
checkpoint_config = dict(interval=1)
log_config = dict(interval=40, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_method = 'fork'
label_convertor = dict(
    type='AttnConvertor', dict_type='DICT90', with_unknown=True)
model = dict(
    type='SARNet',
    backbone=dict(type='ResNet31OCR'),
    encoder=dict(
        type='SAREncoder', enc_bi_rnn=False, enc_do_rnn=0.1, enc_gru=False),
    decoder=dict(
        type='ParallelSARDecoder',
        enc_bi_rnn=False,
        dec_bi_rnn=False,
        dec_do_rnn=0,
        dec_gru=False,
        pred_dropout=0.1,
        d_k=512,
        pred_concat=True),
    loss=dict(type='SARLoss'),
    label_convertor=dict(
        type='AttnConvertor', dict_type='DICT90', with_unknown=True),
    max_seq_len=30)
optimizer = dict(type='Adam', lr=0.000125)
optimizer_config = dict(grad_clip=None)
lr_config = dict(

In [24]:
from mmocr.datasets import build_dataset
from mmocr.models import build_detector
from mmocr.apis import train_detector
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
#mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

/content/mmocr/mmocr/datasets/utils/loader.py:86: UserWarning: HardDiskLoader is deprecated, please use AnnFileLoader instead.
  'AnnFileLoader instead.', UserWarning)
/content/mmocr/mmocr/datasets/utils/loader.py:102: UserWarning: LmdbLoader is deprecated, please use AnnFileLoader instead.
  'AnnFileLoader instead.', UserWarning)
/content/mmocr/mmocr/apis/train.py:86: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  'please set `runner` in your config.', UserWarning)
/content/mmocr/mmocr/apis/utils.py:53: UserWarning: Remove "MultiRotateAugOCR" to support batch inference since samples_per_gpu > 1.
  warnings.warn(warning_msg)
/content/mmocr/mmocr/datasets/utils/loader.py:102: UserWarning: LmdbLoader is deprecated, please use AnnFileLoader instead.
  'AnnFileLoader instead.', UserWarning)
2022-04-27 00:41:17,082 - mmocr - INFO - Start running, host: root@e0466e2ac04e, work_dir: /content/work_train
2022-04-27 00:41:17,083 - mmocr - IN

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 9/9, 5.0 task/s, elapsed: 2s, ETA:     0s

2022-04-27 00:41:59,004 - mmocr - INFO - 
Evaluateing /content/drive/MyDrive/signate/Toppan2/test_ocr/train_text/label.lmdb with 9 images now
2022-04-27 00:41:59,012 - mmocr - INFO - Epoch(val) [1][2]	0_word_acc: 0.0000, 0_word_acc_ignore_case: 0.0000, 0_word_acc_ignore_case_symbol: 0.0000, 0_char_recall: 0.0000, 0_char_precision: 0.0000, 0_1-N.E.D: 0.0000
2022-04-27 00:42:06,912 - mmocr - INFO - Epoch [2][40/225]	lr: 1.250e-04, eta: 0:02:12, time: 0.197, data_time: 0.055, memory: 2004, loss_ce: 0.3493, loss: 0.3493
2022-04-27 00:42:12,661 - mmocr - INFO - Epoch [2][80/225]	lr: 1.250e-04, eta: 0:02:05, time: 0.144, data_time: 0.002, memory: 2004, loss_ce: 0.3374, loss: 0.3374
2022-04-27 00:42:18,405 - mmocr - INFO - Epoch [2][120/225]	lr: 1.250e-04, eta: 0:01:58, time: 0.144, data_time: 0.002, memory: 2004, loss_ce: 0.3284, loss: 0.3284
2022-04-27 00:42:24,140 - mmocr - INFO - Epoch [2][160/225]	lr: 1.250e-04, eta: 0:01:51, time: 0.143, data_time: 0.002, memory: 2004, loss_ce: 0.3367, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 9/9, 14.4 task/s, elapsed: 1s, ETA:     0s

2022-04-27 00:42:37,164 - mmocr - INFO - 
Evaluateing /content/drive/MyDrive/signate/Toppan2/test_ocr/train_text/label.lmdb with 9 images now
2022-04-27 00:42:37,166 - mmocr - INFO - Epoch(val) [2][2]	0_word_acc: 0.0000, 0_word_acc_ignore_case: 0.0000, 0_word_acc_ignore_case_symbol: 0.0000, 0_char_recall: 0.0000, 0_char_precision: 0.0000, 0_1-N.E.D: 0.0000
2022-04-27 00:42:45,086 - mmocr - INFO - Epoch [3][40/225]	lr: 1.250e-04, eta: 0:01:33, time: 0.197, data_time: 0.056, memory: 2004, loss_ce: 0.2897, loss: 0.2897
2022-04-27 00:42:50,839 - mmocr - INFO - Epoch [3][80/225]	lr: 1.250e-04, eta: 0:01:27, time: 0.144, data_time: 0.002, memory: 2004, loss_ce: 0.3289, loss: 0.3289
2022-04-27 00:42:56,590 - mmocr - INFO - Epoch [3][120/225]	lr: 1.250e-04, eta: 0:01:21, time: 0.144, data_time: 0.002, memory: 2004, loss_ce: 0.3186, loss: 0.3186
2022-04-27 00:43:02,340 - mmocr - INFO - Epoch [3][160/225]	lr: 1.250e-04, eta: 0:01:15, time: 0.144, data_time: 0.002, memory: 2004, loss_ce: 0.3058, 

...

2022-04-27 00:43:15,383 - mmocr - INFO - 
Evaluateing /content/drive/MyDrive/signate/Toppan2/test_ocr/train_text/label.lmdb with 9 images now
2022-04-27 00:43:15,387 - mmocr - INFO - Epoch(val) [3][2]	0_word_acc: 0.0000, 0_word_acc_ignore_case: 0.0000, 0_word_acc_ignore_case_symbol: 0.0000, 0_char_recall: 0.0000, 0_char_precision: 0.0000, 0_1-N.E.D: 0.0000
2022-04-27 00:43:23,310 - mmocr - INFO - Epoch [4][40/225]	lr: 1.250e-05, eta: 0:00:58, time: 0.198, data_time: 0.056, memory: 2004, loss_ce: 0.2878, loss: 0.2878
2022-04-27 00:43:29,049 - mmocr - INFO - Epoch [4][80/225]	lr: 1.250e-05, eta: 0:00:53, time: 0.143, data_time: 0.002, memory: 2004, loss_ce: 0.2737, loss: 0.2737
2022-04-27 00:43:34,791 - mmocr - INFO - Epoch [4][120/225]	lr: 1.250e-05, eta: 0:00:47, time: 0.144, data_time: 0.002, memory: 2004, loss_ce: 0.2710, loss: 0.2710
2022-04-27 00:43:40,533 - mmocr - INFO - Epoch [4][160/225]	lr: 1.250e-05, eta: 0:00:41, time: 0.144, data_time: 0.002, memory: 2004, loss_ce: 0.2899, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 9/9, 14.7 task/s, elapsed: 1s, ETA:     0s

2022-04-27 00:43:53,389 - mmocr - INFO - 
Evaluateing /content/drive/MyDrive/signate/Toppan2/test_ocr/train_text/label.lmdb with 9 images now
2022-04-27 00:43:53,393 - mmocr - INFO - Epoch(val) [4][2]	0_word_acc: 0.0000, 0_word_acc_ignore_case: 0.0000, 0_word_acc_ignore_case_symbol: 0.0000, 0_char_recall: 0.0000, 0_char_precision: 0.0000, 0_1-N.E.D: 0.0000
2022-04-27 00:44:01,326 - mmocr - INFO - Epoch [5][40/225]	lr: 1.250e-06, eta: 0:00:26, time: 0.198, data_time: 0.056, memory: 2004, loss_ce: 0.2465, loss: 0.2465
2022-04-27 00:44:07,064 - mmocr - INFO - Epoch [5][80/225]	lr: 1.250e-06, eta: 0:00:20, time: 0.143, data_time: 0.002, memory: 2004, loss_ce: 0.2496, loss: 0.2496
2022-04-27 00:44:12,809 - mmocr - INFO - Epoch [5][120/225]	lr: 1.250e-06, eta: 0:00:14, time: 0.144, data_time: 0.002, memory: 2004, loss_ce: 0.2469, loss: 0.2469
2022-04-27 00:44:18,542 - mmocr - INFO - Epoch [5][160/225]	lr: 1.250e-06, eta: 0:00:09, time: 0.143, data_time: 0.002, memory: 2004, loss_ce: 0.2675, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 9/9, 14.5 task/s, elapsed: 1s, ETA:     0s

2022-04-27 00:44:31,449 - mmocr - INFO - 
Evaluateing /content/drive/MyDrive/signate/Toppan2/test_ocr/train_text/label.lmdb with 9 images now
2022-04-27 00:44:31,453 - mmocr - INFO - Epoch(val) [5][2]	0_word_acc: 0.0000, 0_word_acc_ignore_case: 0.0000, 0_word_acc_ignore_case_symbol: 0.0000, 0_char_recall: 0.0000, 0_char_precision: 0.0000, 0_1-N.E.D: 0.0000


## 学習結果の確認

In [25]:
%cd /content/mmocr

/content/mmocr


In [26]:

from mmocr.apis import init_detector, model_inference

img = #確認する画像ファイルのパス
checkpoint = # 利用するモデルのパス
out_file = # 出力するファイルのパス

model = init_detector(cfg, checkpoint, device=device)
if model.cfg.data.test['type'] == 'ConcatDataset':
    model.cfg.data.test.pipeline = model.cfg.data.test['datasets'][0].pipeline

result = model_inference(model, img)
print(f'result: {result}')

img = model.show_result(
        img, result, out_file=out_file, show=False)



load checkpoint from local path: /content/work_train/epoch_5.pth


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


result: {'text': '<UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN><UKN>', 'score': 0.19964126575560798}
